In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Directory to store LSB pictures

In [ ]:
!mkdir /kaggle/working/lsb

# Method to hide a message in an image

- simple algorithm, you can control the number of bits per channel (each pixel has 3 channels)
- chooses a random starting point (such that the message can fit in the image) and a random 'skip' amount to skip across pixels at a specific interval

In [ ]:
def hide_message(img, message, num_bits_per_channel=1, verbose=False):
    
    pixels = list(img.getdata())    
    binary_data_string = ''.join(format(ord(char), '08b') for char in message)    
    random_skip = random.randint(0, 10000)
    num_pixels_to_use = math.ceil(len(binary_data_string) / num_bits_per_channel / 3) * random_skip # 3 channels
    
    while num_pixels_to_use >= len(pixels) - 2:
        random_skip = random.randint(0, 8000)
        num_pixels_to_use = math.ceil(len(binary_data_string) / num_bits_per_channel / 3) * random_skip # 3 channels
    
    if verbose:
        print(f"bin_data_len: {len(binary_data_string)} skip: {random_skip}, num_pixels_to_use: {num_pixels_to_use}")
    
    data_index = 0
    pixel_index = random.randint(0, len(pixels) - 1 - num_pixels_to_use)       
    
    if verbose:
        print(f"start pixel index: {pixel_index}")
        
    skip = random_skip
        
    while data_index < len(binary_data_string):
        pixel = pixels[pixel_index]
        new_pixel = list(pixel)
        if verbose:
            print(f"orig: {new_pixel} at index {pixel_index}")
        for channel in range (3):
    #         if data_index >= len(binary_data_string):
    #             break;
            if verbose:
                print(f'channel {channel}')
                print(f"orig: {format(new_pixel[channel], 'b')}")
            new_pixel[channel] = pixel[channel] & ~(2**num_bits_per_channel - 1)
            if verbose:
                print(f"masked: {format(new_pixel[channel], 'b')}")
            for j in range(num_bits_per_channel):
                if data_index >= len(binary_data_string):
                    break;
                new_pixel[channel] |= (int(binary_data_string[data_index]) << j)
                if verbose:
                    print(f'adding bit {int(binary_data_string[data_index])} to position {j}')
                data_index += 1
            if verbose:
                print(f"final: {format(new_pixel[channel], 'b')}")
        pixels[pixel_index] = tuple(new_pixel)
        pixel_index += skip
        if verbose:
            print("new pixel: ", new_pixel);
            print("-" * 10)
    
    return pixels

In [ ]:
# import numpy as np
# import random

# def hide_message(img, message, num_bits_per_channel=1, verbose=False):
#     pixels = np.array(img, dtype=np.uint8)  # Ensure pixels are of type uint8
#     binary_data_string = ''.join(format(ord(char), '08b') for char in message)
    
#     # Calculate the number of pixels to use outside the loop
#     random_skip = random.randint(0, 10000)
#     num_pixels_to_use = math.ceil(len(binary_data_string) / num_bits_per_channel / 3) * random_skip

#     while num_pixels_to_use >= pixels.size - 2:
#         random_skip = random.randint(0, 8000)
#         num_pixels_to_use = math.ceil(len(binary_data_string) / num_bits_per_channel / 3) * random_skip

#     if verbose:
#         print(f"bin_data_len: {len(binary_data_string)} skip: {random_skip}, num_pixels_to_use: {num_pixels_to_use}")

#     data_index = 0

#     # Calculate indices outside the loop
#     pixel_indices = np.arange(random.randint(0, pixels.size - 1 - num_pixels_to_use), step=random_skip)

#     if verbose:
#         print(f"start pixel index: {pixel_indices[0]}")

#     for pixel_index in pixel_indices:
#         if data_index >= len(binary_data_string):
#             break

#         # Perform bitwise operations using NumPy
#         pixels_flat = pixels.flatten().astype(np.uint16)  # Convert to uint16 to avoid casting issues
#         pixels_flat[pixel_index:pixel_index+3] &= ~(2**num_bits_per_channel - 1)

#         for j in range(num_bits_per_channel):
#             if data_index >= len(binary_data_string):
#                 break
#             pixels_flat[pixel_index:pixel_index+3] |= (int(binary_data_string[data_index]) << j)
#             data_index += 1

#     # Reshape back to the original shape and convert back to uint8
#     pixels = pixels_flat.reshape(img.size[::-1] + (3,)).astype(np.uint8)

#     return pixels


# Sample Run + Plotting

In [ ]:
from PIL import Image
import random
import math

# 
msg = "AAA"
with open('/kaggle/input/dialogues-and-scripts/StarWars_EpisodeIV_script.txt', 'rb') as file:
    try:
        content = file.read().decode('utf-8', errors='ignore')
        msg = content
    except UnicodeDecodeError as e:
        print(f"UnicodeDecodeError: {e}")

img = Image.open('/kaggle/input/alaska2-image-steganalysis/Cover/00001.jpg')

pixels = hide_message(img, msg, num_bits_per_channel=5, verbose=False)
    
encoded_image_path = '/kaggle/working/test.jpg'
encoded_img = Image.new(img.mode, img.size)
encoded_img.putdata(pixels)
encoded_img.save(encoded_image_path, quality=100)

arr1 = np.array(img)
arr2 = np.array(encoded_img)
diff = arr1 - arr2

# print(np.where(arr1 != arr2))

feature_vector1 = arr1.flatten()
feature_vector2 = arr2.flatten()
feature_vector3 = diff.flatten()

# print(np.where(feature_vector1 != feature_vector2))

# Concatenate feature vectors
feature_matrix = np.vstack((feature_vector1, feature_vector2, feature_vector3))

print(feature_matrix.shape)
    
import matplotlib.pyplot as plt

# Reshape the feature matrix to (num_images, height, width, channels)
feature_matrix = feature_matrix.reshape((3, 512, 512, 3))

# Row by column grid
fig, axes = plt.subplots(1, 3)

# Show Cover, Stego, and Cover - Stego images
axes[0].imshow(feature_matrix[0])
axes[0].axis('off')  # Turn off axis labels
axes[1].imshow(feature_matrix[1])
axes[1].axis('off')  
axes[2].imshow(feature_matrix[2])
axes[2].axis('off')  

plt.show()

# Read in scripts, hide in all 'Cover' images from ALASKA2, resize to 28x28 

In [ ]:
# read in the scripts to use as messages
root = '/kaggle/input/dialogues-and-scripts'

input_file_paths = os.listdir(root)
messages = []     

for path in input_file_paths:
    full_path = os.path.join(root, path)
    print(full_path)
    with open(full_path, 'rb') as file:
        try:
            content = file.read().decode('utf-8', errors='ignore')
            messages.append(content)
        except UnicodeDecodeError as e:
            print(f"UnicodeDecodeError: {e}")

num_bits_per_channel = 1
i = 1

# def hide_images(paths):

# import threading

# threads = []

# for i in range(20):
#     t = threading.Thread(target=print_hello, args=(f'Thread {i}',))
#     threads.append(t)

# for t in threads:
#     t.start()

# for t in threads:
#     t.join()
    

for cover_image_path in os.listdir('/kaggle/input/alaska2-image-steganalysis/Cover/'):
    full_path = os.path.join('/kaggle/input/alaska2-image-steganalysis/Cover/', cover_image_path)

    img = Image.open(full_path)
    img.resize((28, 28))
    
    # choose a random message from a random script
    message_index = i % len(messages)
    message_to = random.randint(5000, len(messages[message_index]))    
    message = messages[message_index][message_to - (100 * ((i % 4) + 1)):message_to]

#     message = 'ABCERDKJDFKJ'

    pixels = hide_message(img, message, num_bits_per_channel=num_bits_per_channel, verbose=False)

    encoded_image_path = os.path.join('/kaggle/working/lsb', cover_image_path)
    encoded_img = Image.new(img.mode, img.size)
    encoded_img.putdata(pixels)
    encoded_img.save(encoded_image_path, quality=100)
    if i % 10 == 0:
        print(f'saved at {encoded_image_path}')

    num_bits_per_channel = ((num_bits_per_channel + 1) % 6) + 1

    if i % 1000 == 0:
        print(f"length: {i}")
    i += 1


# def decode_msg(img, message, num_bits_per_channel=1, verbose=False):
# print('decoding')
# print('-' * 10)
# recovered = ''
# pixel_index = 0
# data_index = 0

# while data_index < len(binary_data_string):
#     pixel = pixels[pixel_index]
#     for channel in range(3):  
#         print(f'channel {channel}')
#         print(f"pixel: {format(new_pixel[channel], 'b')}")
#         for shift in range (num_bits_per_channel): # if num_bits_per_channel is 2 -> range is 1, 0
#             if data_index >= len(binary_data_string):
#                 break;
#             bit = (pixel[channel] & (1 << shift)) >> shift
#             print(f'extracted bit {bit} frim position {shift}')
#             recovered += str(bit)
#             data_index += 1
#     pixel_index += 1
# print(binary_data_string)
# print(recovered)
# print(binary_data_string == recovered)